In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import yaml
from tqdm.notebook import tqdm
import galsim
import batoid
import wfsim
import matplotlib.pyplot as plt

In [ ]:
# Some initial setup
# We'll do r-band for this demo.
bandpass = galsim.Bandpass("LSST_r.dat", wave_type='nm')
telescope = batoid.Optic.fromYaml("LSST_r.yaml")
pixel_scale = 10e-6

In [ ]:
bandpass.effective_wavelength

In [ ]:
# Setup observation parameters.  Making ~plausible stuff up.
observation = {
    'zenith': 30 * galsim.degrees,
    'raw_seeing': 0.7 * galsim.arcsec,  # zenith 500nm seeing
    'wavelength': bandpass.effective_wavelength,
    'exptime': 15.0,  # seconds
    'temperature': 293.,  # Kelvin
    'pressure': 69.,  #kPa
    'H2O_pressure': 1.0  #kPa
}

In [ ]:
# Setup atmospheric parameters
atm_kwargs = {
    'screen_size': 819.2,
    'screen_scale': 0.1,
    'nproc': 6  # create screens in parallel using this many CPUs
}

In [ ]:
# We loaded the fiducial telescope, but we actually want to perturb it 
# out of alignment a bit and misfigure the mirrors a bit.
# The big question is how much to perturb each potential 
# degree-of-freedom.  Let's not dwell on that at the moment though; for
# demonstration, the following will do.

rng = np.random.default_rng()

# Misalignments of M2 and camera first
M2_offset = np.array([
    rng.uniform(-0.0001, 0.0001),  # meters
    rng.uniform(-0.0001, 0.0001),
    rng.uniform(-0.00001, 0.00001),
])    
M2_tilt = (
    batoid.RotX(np.deg2rad(rng.uniform(-0.01, 0.01)/60)) @
    batoid.RotY(np.deg2rad(rng.uniform(-0.01, 0.01)/60))
)

camera_offset = np.array([
    rng.uniform(-0.001, 0.001),  # meters
    rng.uniform(-0.001, 0.001),
    rng.uniform(-0.00001, 0.00001),    
])
camera_tilt = (
    batoid.RotX(np.deg2rad(rng.uniform(-0.01, 0.01)/60)) @
    batoid.RotY(np.deg2rad(rng.uniform(-0.01, 0.01)/60))
)

perturbed = (
    telescope
    .withGloballyShiftedOptic("M2", M2_offset)
    .withLocallyRotatedOptic("M2", M2_tilt)
    .withGloballyShiftedOptic("LSSTCamera", camera_offset)
    .withLocallyRotatedOptic("LSSTCamera", camera_tilt)
)

# Now let's perturb the mirrors, we should use the actual mirror modes
# here, but for now we'll just use Zernike polynomials.  

M1M3_modes = rng.uniform(-0.05, 0.05, size=25) # waves
M1M3_modes *= bandpass.effective_wavelength*1e-9 # -> meters
# M1M3 bends coherently, so use a single Zernike perturbation for both,
# Set the outer radius to the M1 radius so the polynomial doesn't 
# explode.  It's fine to use a circular Zernike here though (no inner 
# radius).

M1M3_surface_perturbation = batoid.Zernike(
    M1M3_modes,
    R_outer=telescope['M1'].obscuration.original.outer,
)
perturbed = perturbed.withSurface(
    "M1",
    batoid.Sum([
        telescope['M1'].surface,
        M1M3_surface_perturbation
    ])
)
perturbed = perturbed.withSurface(
    "M3",
    batoid.Sum([
        telescope['M3'].surface,
        M1M3_surface_perturbation
    ])
)

# M2 gets independent perturbations from M1M3
M2_modes = rng.uniform(-0.05, 0.05, size=25) # waves
M2_modes *= bandpass.effective_wavelength*1e-9 # -> meters

M2_surface_perturbation = batoid.Zernike(
    M2_modes,
    R_outer=telescope['M2'].obscuration.original.outer,
)
perturbed = perturbed.withSurface(
    "M2",
    batoid.Sum([
        telescope['M2'].surface,
        M2_surface_perturbation
    ])
)

In [ ]:
# We can take a quick look at how we've perturbed the optics by making 
# a spot diagram.  The batoid.spot tool returns points in meters, so
# we divide by pixel_scale to get pixels.  We also look in a few points
# around the field of view to get a global picture.

for thx, thy in [(0,0), (-1.5, 0), (1.5, 0), (0, -1.5), (0, 1.5)]:
    sx, sy = batoid.spot(
        perturbed, 
        np.deg2rad(thx), np.deg2rad(thy), 
        bandpass.effective_wavelength*1e-9, 
        nx=128
    )
    plt.figure()
    plt.scatter(sx/pixel_scale, sy/pixel_scale, s=1)
    plt.show()

In [ ]:
simulator = wfsim.SimpleSimulator(
    observation,
    atm_kwargs,
    perturbed,
    bandpass,
    shape=(4096, 4096),
    rng=rng
)

In [ ]:
catalog = wfsim.MockStarCatalog()

In [ ]:
def corners():
    import lsst.sphgeom as sphgeom
    center = sphgeom.UnitVector3d(1, 0, 0)
    # Get a random direction for a corner:
    axis1 = sphgeom.UnitVector3d.orthogonalTo(center)
    axis2 = sphgeom.UnitVector3d.orthogonalTo(center, axis1)
    angle = sphgeom.Angle.fromDegrees(4200*0.2/3600/2)
    bottom = center.rotatedAround(axis1, -angle)
    top = center.rotatedAround(axis1, angle)
    corner1 = bottom.rotatedAround(axis2, -angle)    
    corner2 = bottom.rotatedAround(axis2, angle)    
    corner3 = top.rotatedAround(axis2, -angle)    
    corner4 = top.rotatedAround(axis2, angle)    
    poly = sphgeom.ConvexPolygon([corner1, corner2, corner3, corner4])
    return poly

In [ ]:
xyz, mag = catalog.get_stars(corners())
ra = np.arctan2(xyz[:,1], xyz[:,0])
dec = np.arcsin(xyz[:,2], np.hypot(xyz[:,0], xyz[:,1]))
phot = (32.36 * 15 * 10**(-0.4*(mag-24.0))).astype(int)  # Using WeakLensingDeblending zeropoints
print(f"{np.sum(phot):_d} photons")

In [ ]:
with tqdm(total=np.sum(phot), unit_scale=True) as pbar:
    for ra_, dec_, phot_ in zip(ra, dec, phot):
        star_T = rng.uniform(4000, 10000)
        sed = wfsim.BBSED(star_T)
        simulator.add_star(ra_, dec_, sed, phot_, rng)
        pbar.update(phot_)

In [ ]:
skymag = 20.5  # Also WLD value
skyphot_arcsec = (32.36 * 15 * 10**(-0.4*(skymag-24.0)))
skyphot_pixel = skyphot_arcsec * 0.2**2

In [ ]:
simulator.add_background(skyphot_pixel, rng)

In [ ]:
%matplotlib widget
plt.figure(figsize=(7, 7))
plt.imshow(simulator.image.array, vmin=-30, vmax=300, cmap='Greys_r')
plt.show()

In [ ]:
# Repeat with intrafocal
intra_telescope = perturbed.withGloballyShiftedOptic("Detector", [0,0,-0.0015])

In [ ]:
intra_simulator = wfsim.SimpleSimulator(
    observation,
    atm_kwargs,
    intra_telescope,
    bandpass,
    shape=(4096, 4096),
    rng=rng
)

In [ ]:
with tqdm(total=np.sum(phot), unit_scale=True) as pbar:
    for ra_, dec_, phot_ in zip(ra, dec, phot):
        star_T = rng.uniform(4000, 10000)
        sed = wfsim.BBSED(star_T)
        intra_simulator.add_star(ra_, dec_, sed, phot_, rng)
        pbar.update(phot_)

In [ ]:
intra_simulator.add_background(skyphot_pixel, rng)

In [ ]:
plt.figure(figsize=(7, 7))
# plt.imshow(intra_simulator.image.array, vmin=-30, vmax=300, cmap='Greys_r')
plt.imshow(np.arcsinh(intra_simulator.image.array), vmin=-3, vmax=9, cmap='Greys_r')
plt.show()